# Building The General Model

Using the analysis created by the previous sections, we will try to build and solve the
general model for concurrency value autoscaling.


One important thing to note here is that the general model is evaluated every 2 seconds
because of the inherent autoscaling evaluation period of 2 seconds in knative. As a result
we need to convert the rates of provisioning and deprovisioning of containers from
time rates into probability rates (from CTMC to DTCM parameters).

In [1]:
%load_ext autoreload
%autoreload 2
# imports

# important libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy as sp

from tqdm.auto import tqdm

# for better printing of variables
from IPython.display import display

# custom imports
from concperf import single_model, general_model
from concperf import utility

In [2]:
# update configuration dictionary for each instance count
def update_config(config):
    config['arrival_rate_server'] = config['arrival_rate_total'] / config['instance_count']
    config['base_service_time'] = config['base_service_time_ms'] / 1000

model_config = {
    # 'instance_count' should be added for each state
    'max_conc': 10,
    'arrival_rate_total': 5,
    'alpha': 0.11,
    'base_service_time_ms': 1154,
    'max_container_count': 25,
    'target_conc': 0.7, # assumes target utilization
    'max_scale_up_rate': 1000, # from N to 1000*N at most
    'max_scale_down_rate': 2, # from N to N/2 at most
    'stable_conc_avg_count': 300, # number of times monitored concurrency will be averaged in stable mode
    'autoscaling_interval': 2, # amount of time between autoscaling evaluations
    'provision_rate_base': 1,
    'deprovision_rate_base': 2,
}

single_coder = single_model.StateCoder(config=model_config)

In [3]:
general_state_coder = general_model.StateCoder(model_config)
print('Number of states:', general_state_coder.get_state_count())
display(general_state_coder.get_state_list()[:10])

Number of states: 676


[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9)]

In [4]:
solution = utility.get_trans_probs(10, transition_rate_base=1, max_t=model_config['autoscaling_interval'])
solution

array([0.13533528, 0.11701964, 0.10118276, 0.08748916, 0.07564879,
       0.06541084, 0.05655845, 0.04890409, 0.04228564, 0.27016534])

In [5]:
# find parameters for different configuration

# TODO: do something about instance count of zero

for inst_count in range(1, model_config['max_container_count']+1):
    # add instance count to config
    model_config.update({
        'instance_count': inst_count,
    })

    # update the config
    update_config(model_config)

    # calculate and show Q
    Q = single_model.get_single_container_q(single_coder, config=model_config)
    # display(pd.DataFrame(Q))

    req_count_prob = utility.solve_CTMC(Q)
    req_df = pd.DataFrame(data = {
        'req_count': [s[0] for s in single_coder.get_state_list()],
        'req_count_prob': req_count_prob,
    })

    # calculate measure concurrency distribution
    avg_count = model_config['stable_conc_avg_count']
    import time
    start_time = time.time()
    req_count_averaged_vals, req_count_averaged_probs = utility.get_averaged_distribution(vals=req_df['req_count'], probs=req_df['req_count_prob'], avg_count=avg_count)
    print(f"Calculation took {time.time() - start_time} seconds for {inst_count} instances")

    # calculate probability of different ordered instance count
    new_order_val, new_order_prob = general_model.get_new_order_dist(req_count_averaged_vals, req_count_averaged_probs, model_config)

    # plot the result
    # plt.figure(figsize=(8,4))
    # plt.bar(new_order_val, new_order_prob, width=1)

Calculation took 1.0576083660125732 seconds for 1 instances
Calculation took 1.1412718296051025 seconds for 2 instances
Calculation took 0.8463587760925293 seconds for 3 instances
Calculation took 0.6916096210479736 seconds for 4 instances
Calculation took 0.5939676761627197 seconds for 5 instances
Calculation took 0.5223915576934814 seconds for 6 instances
Calculation took 0.47178196907043457 seconds for 7 instances
Calculation took 0.46110963821411133 seconds for 8 instances
Calculation took 0.4214193820953369 seconds for 9 instances
Calculation took 0.4008769989013672 seconds for 10 instances
Calculation took 0.3917732238769531 seconds for 11 instances
Calculation took 0.3611767292022705 seconds for 12 instances
Calculation took 0.33937644958496094 seconds for 13 instances
Calculation took 0.33635544776916504 seconds for 14 instances
Calculation took 0.32183218002319336 seconds for 15 instances
Calculation took 0.30989789962768555 seconds for 16 instances
Calculation took 0.30310058